<a href="https://colab.research.google.com/github/brianchirn/askdocs/blob/main/askdocs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code will pull the top 25 posts from the subreddit /r/askdocs and then try to use Natural Language Processing to summarize the post. 

Request use and obtain keys via https://www.reddit.com/prefs/apps 

In [ ]:
CLINET_ID = 'ysMKupnzIJk_hw' #personal use script
SECRET_KEY = 'Dfe-vumHwUhs_rxUBulZYmZ3R4vb4Q'


Request a temporary off token from reddit

In [ ]:
import requests
auth = requests.auth.HTTPBasicAuth(CLINET_ID, SECRET_KEY)

Login in to reddit with a dictionary

In [ ]:
data = {
    'grant_type' : 'password',
    'username' : '_123test',
    'password' : 'password'
}

headers = {'User-Agent': 'MyAPI/0.0.1'}

Send a request for our auth token.
We are requesting the version 1 of the api, the access_token

We need to include our auth, data and headers




In [ ]:
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth = auth, data = data, headers = headers)


In [ ]:
res.json()

In [ ]:
TOKEN = res.json()['access_token']
#reformat headers
headers = {**headers, **{'Authorization': f'bearer {TOKEN}'}}

In [ ]:
headers

We are now able to access API on reddit as long as we have headers = headers

Now let's access all of the hot posts within a subreddit


In [ ]:
res = requests.get('https://oauth.reddit.com/r/askdocs/top', 
                 headers = headers, params = {'limit': 25})

In [ ]:
for post in res.json()['data']['children']:
  print(post['data'])

Let's convert this into pandas dataframe


In [ ]:
import pandas as pd
df = pd.DataFrame()

for post in res.json()['data']['children']:
  df = df.append({
      'subreddit' : post['data']['subreddit'],
      'title' : post['data']['title'],
      'selftext' : post['data']['selftext'],
      'upvote_ratio': post['data']['upvote_ratio'],
      'ups': post['data']['ups'],
      'length': len(post['data']['selftext'])

  }, ignore_index = True)

In [ ]:
post['data'].keys() #see what info you want to extract from post
df


In [ ]:
import re
test = df['title'][14]+ df['selftext'][14]
test = (re.sub('[()]', '', test)) #remove parathesis
test = re.sub(r"[\[\]]", "", test) #remove brackets

Using NLTK to summarize text 
code taking from here:
https://stackabuse.com/text-summarization-with-nltk-in-python/

In [ ]:
import nltk
nltk.download('punkt')
sentence_list = nltk.sent_tokenize(test)
sentence_list

In [ ]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
word_frequencies = {}

#modify code to weigh numbers higher than normal words
def hasNumbers(inputString):
  return any(char.isdigit() for char in inputString)

for word in nltk.word_tokenize(test):
  if word not in stopwords:
    if word not in word_frequencies.keys():
      if hasNumbers(word) == True:
        word_frequencies[word] = 5
      else:
        word_frequencies[word] = 1
    else:
      word_frequencies[word] += 1
print(word_frequencies)

In [ ]:
maximum_frequency = max(word_frequencies.values())
for word in word_frequencies.keys():
  word_frequencies[word] = (word_frequencies[word]/maximum_frequency)
  

In [ ]:
sentence_scores = {}
for sent in sentence_list:
  for word in nltk.word_tokenize(sent.lower()):
    if word in word_frequencies.keys():
      if len(sent.split(' ')) < 30:
        if sent not in sentence_scores.keys():
          sentence_scores[sent] = word_frequencies[word]
        else:
          sentence_scores[sent]+= word_frequencies[word]



In [ ]:
import heapq
summary_sentences = heapq.nlargest(7, sentence_scores, key = sentence_scores.get)
print('SUMMARY')
print(len(summary))
summary = '\n'.join(summary_sentences)
print(summary)



print('\nORIGINAL')
print(len(test))
test